# this is a script for eccentricity

# Read Data

In [1]:
% ERA5 SAT&SIC 1980-2020
load('/glade/campaign/univ/ufsu0029/work/Data/ERA5/t2m_8020.mat');
load('/glade/campaign/univ/ufsu0029/work/Data/ERA5/sic_8020.mat');
lon2=lon;lat2=lat;

In [2]:
load('/glade/campaign/univ/ufsu0029/work/Data/ERA5/meridional_velocity.mat');

In [3]:
% NCEP2 SAT 1980-2020
load('/glade/campaign/univ/ufsu0029/work/Data/NCEP_DOE/air2m_8020.mat');
lon3=lon;lat3=lat;

In [4]:
% CESM2 SAT&solar
load('/glade/campaign/univ/ufsu0029/work/Data/eccen/t2m_m.mat');
load('/glade/campaign/univ/ufsu0029/work/Data/eccen/solar_m.mat');
solar_m=squeeze(mean(solar_m,1));
lon4=lon;lat4=lat;

In [5]:
curr_path='/glade/campaign/univ/ufsu0029/work/MATLAB/annual_cycle/eccentricity/submit/';

# Example for Asymmetry

## semiannual&annual component in solar

In [15]:
sr=squeeze(solar_m([65,17],:,1));  % 30N, 60S
lat0=lat4;

aa1=sr;
aa1=aa1-mean(aa1,2);
aa=mean(aa1.*exp(1i*2*pi*reshape(1:2,1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:2,1,1,[]).*[.5:365]/365));
ex_sr=aa1;
ex_sr(:,:,2:3)=aa;
ex_sr(2,:,:)=ex_sr(2,[183:end,1:182],:);

save(append(curr_path,'/ex_sr.mat'),'ex_sr');

In [16]:
sr=squeeze(solar_m(:,:,1));
lat0=lat4;

aa1=sr;
aa1=aa1-mean(aa1,2);
aa=mean(aa1.*exp(1i*2*pi*reshape(1:2,1,1,[]).*[.5:365]/365),2);
sr_an_sm=2*real(aa.*exp(-1i*2*pi*reshape(1:2,1,1,[]).*[.5:365]/365));

lat0=(lat0([1,1:end])+lat0([1:end,end]))/2;
lat0(1)=2*lat0(1)-lat0(2);
lat0(end)=2*lat0(end)-lat0(end-1);
day=[0:365]';
save(append(curr_path,'sr_an_sm.mat'),'sr_an_sm','lat0','day');

## Constant D

### 50m

In [17]:
sr=squeeze(solar_m([65,17],:,1));
sr(2,:)=sr(2,[183:end,1:182]);
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1

dt=6*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:364.9]; %date in a year

D=50*(1+.0*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.4*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [18]:
aa1=To(:,end-365*4+2:4:end);
aa1=aa1-mean(aa1,2);
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:4),3);
aa1(:,:,2:3)=aa(:,:,1:2);
[~,imax]=max(aa1(:,:,1:2),[],2);imax=imax/24;
[~,imin]=min(aa1(:,:,1:2),[],2);imin=imin/24;
mm=imax;
mm(:,2,:)=imin;
imax(imax<imin)=imax(imax<imin)+365;
mm(:,3,:)=365-2*(imax-imin);
TT=aa1(:,12:24:end,:);

save(append(curr_path,'full_50.mat'),'TT','mm');

### 2m

In [19]:
sr=squeeze(solar_m([65,17],:,1));
sr(2,:)=sr(2,[183:end,1:182]);
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1

dt=6*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:364.9]; %date in a year

D=2*(1+.0*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.5*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [20]:
aa1=To(:,end-365*4+2:4:end);
aa1=aa1-mean(aa1,2);
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:4),3);
aa1(:,:,2:3)=aa(:,:,1:2);
[~,imax]=max(aa1(:,:,1:2),[],2);imax=imax/24;
[~,imin]=min(aa1(:,:,1:2),[],2);imin=imin/24;
mm=imax;
mm(:,2,:)=imin;
imax(imax<imin)=imax(imax<imin)+365;
mm(:,3,:)=365-2*(imax-imin);
TT=aa1(:,12:24:end,:);

save(append(curr_path,'full_2.mat'),'TT','mm');

## Seasonal D Example

### 30N

In [21]:
sr=reshape(solar_m(65,:,1),1,[]); % 30N, 60S
lat0=lat4;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.2;gamma=0;epsilon=1; %1

dt=6*60*60;tt=50*365*4;
t2=[0:(1/4):364.9]; %date in a year

D=50*(1+.6*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr; % 29.5N--32, 29.5S--63
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

To=zeros(size(sr1,1),1000,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),1,tt);
aa2=zeros(size(sr1,1),1,tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,1,m)=(1-alpha)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4;
    aa2(:,1,m)=gamma*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4;
    
    if(m<5.5*365*4)
        To(:,:,m+1)=To(:,:,m)+dt*aa1(:,1,m)/4e7;
        Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,1,m)/(Cpa*ma);
    else
        To(:,1:DD,m+1)=0*To(:,1:DD,1)+ ...
            (sum(To(:,1:DD,m),2)*Cpo*rhoodz+dt*aa1(:,1,m))/(DD*Cpo*rhoodz);
        To(:,DD+1:end,m+1)=To(:,DD+1:end,m);
        Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,1,m)/(Cpa*ma);
    end
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [22]:
aa1=reshape(To(:,1,end-365*4+2:4:end),1,[]);
aa1=aa1-mean(aa1,2);
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:10),3);
aa1(:,:,2:3)=aa(:,:,1:2);
[~,imax]=max(aa1(:,:,1:2),[],2);imax=imax/24;
[~,imin]=min(aa1(:,:,1:2),[],2);imin=imin/24;
mm=imax;
mm(:,2,:)=imin;
imax(imax<imin)=imax(imax<imin)+365;
mm(:,3,:)=365-2*(imax-imin);
TT=aa1(:,12:24:end,:);

DD=D(1:4:end);
save(append(curr_path,'seasonal_30N.mat'),'TT','mm','DD');

### 60S

In [23]:
sr=reshape(solar_m(17,:,1),1,[]); % 60S
sr=sr(1,[183:end,1:182]);
lat0=lat4;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.2;gamma=0;epsilon=1; %1

dt=6*60*60;tt=50*365*4;
t2=[0:(1/4):364.9]; %date in a year

D=50*(1-.6*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

To=zeros(size(sr1,1),1000,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),1,tt);
aa2=zeros(size(sr1,1),1,tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,1,m)=(1-alpha)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4;
    aa2(:,1,m)=gamma*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4;
    
    if(m<10*365*4)
        To(:,:,m+1)=To(:,:,m)+dt*aa1(:,1,m)/4e7;
        Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,1,m)/(Cpa*ma);
    else
        To(:,1:DD,m+1)=0*To(:,1:DD,1)+ ...
            (sum(To(:,1:DD,m),2)*Cpo*rhoodz+dt*aa1(:,1,m))/(DD*Cpo*rhoodz);
        To(:,DD+1:end,m+1)=To(:,DD+1:end,m);
        Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,1,m)/(Cpa*ma);
    end
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [24]:
aa1=reshape(To(:,1,end-365*4+2:4:end),1,[]);
aa1=aa1-mean(aa1,2);
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:4),3);
aa1(:,:,2:3)=aa(:,:,1:2);
[~,imax]=max(aa1(:,:,1:2),[],2);imax=imax/24;
[~,imin]=min(aa1(:,:,1:2),[],2);imin=imin/24;
mm=imax;
mm(:,2,:)=imin;
imax(imax<imin)=imax(imax<imin)+365;
mm(:,3,:)=365-2*(imax-imin);
TT=aa1(:,12:24:end,:);

DD=D(1:4:end);
save(append(curr_path,'seasonal_60S.mat'),'TT','mm','DD');

# Northward Air Mass Flux

In [3]:
R=6370000;dp=10000;g=10;lat=lat2;

ms_n=reshape(mean(mean(v_n,1),3).*cosd(lat')*2*pi*R,[],1); %kg/s
ms_s=reshape(mean(mean(v_s,1),3).*cosd(lat')*2*pi*R,[],1);

eq_n=reshape(mean(mean(v_n,1),3)/100000*g,[],1);
eq_s=reshape(mean(mean(v_s,1),3)/100000*g,[],1);

save(append(curr_path,'mass_flux.mat'),'ms_n','ms_s','eq_n','eq_s','lat');

# Eccentricity

## insolation omega=100&280

In [1]:
aa1=solar_m;
lat=lat4;

aa1=aa1-mean(aa1,2);
aa=mean(aa1.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(.5:365,1,[])/365),2);
aa2=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(.5:365,1,[])/365));
sa_amp=squeeze(abs(aa(:,:,:,2)));

save(append(curr_path,'sol_rt_280_000_100.mat'),'sa_amp','lat');

## ERA5&NCEP delta_T

In [27]:
aa0=t2m; %1980-2020
aa0=reshape(aa0,size(aa0,1),size(aa0,2),[]);
tt=15;aa=0*aa0;
for m=1:2*tt+1
    aa(:,:,tt+1:end-tt)=aa(:,:,tt+1:end-tt)+aa0(:,:,m:end-(2*tt+1-m));
end
aa0=aa/(2*tt+1);
aa0=reshape(aa0,size(aa0,1),size(aa0,2),365,[]);

In [28]:
aa1=squeeze(mean(mean(aa0(:,:,:,2:end-1),1),4));
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:4),3);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_era=365-2*(imax-imin);
lat=lat2;lat(65:117)=nan;

save(append(curr_path,'deltaT_zonal_ERA.mat'),'deltaT_era','lat');

In [29]:
aa0=air2m; %1980-2020
aa0=reshape(aa0,size(aa0,1),size(aa0,2),[]);
tt=15;aa=0*aa0;
for m=1:2*tt+1
    aa(:,:,tt+1:end-tt)=aa(:,:,tt+1:end-tt)+aa0(:,:,m:end-(2*tt+1-m));
end
aa0=aa/(2*tt+1);
aa0=reshape(aa0,size(aa0,1),size(aa0,2),365,[]);

In [30]:
aa1=squeeze(mean(mean(aa0(:,:,:,2:end-1),1),4));
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:4),3);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_ncep=365-2*(imax-imin);
lat=lat3;lat(35:60)=nan;

save(append(curr_path,'deltaT_zonal_NCEP.mat'),'deltaT_ncep','lat');

## simulations

### ocean, MT=off, omega=100

In [31]:
sr=squeeze(solar_m(:,:,3));
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1
c_exc=0; %m/s

dt=6*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:364.9]; %date in a year

D=50*(1+.0*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[1:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

C_EXC=zeros(size(sr1,1),size(sr1,1));
ll=(lat0(1:end-1)+lat0(2:end))/2;
for m=2:size(sr1,1)-1
    C_EXC(m,m-1)=-cosd(ll(m-1));
    C_EXC(m,m)=cosd(ll(m-1))+cosd(ll(m));
    C_EXC(m,m+1)=-cosd(ll(m));
end
C_EXC(1,1)=cosd(ll(1));C_EXC(1,2)=-cosd(ll(1));
C_EXC(end,end-1)=-cosd(ll(end));C_EXC(end,end)=cosd(ll(end));
ll=cosd(lat0);ll([1,end])=dy/8/R;
C_EXC=c_exc*dt/dy*C_EXC./ll;
C_EXC=inv(eye(size(sr1,1))+C_EXC);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.4*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    Ta(:,1,m+1)=C_EXC*Ta(:,1,m+1);
    
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [32]:
aa1=squeeze(To(:,1,end-4*365+1:1:end));
aa=mean(aa1.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(.125:.25:365,1,[])/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(1/48:1/24:365,1,[])/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_50_off_100=365-2*(imax-imin);
lat=lat0;lat(35:62)=nan;

save(append(curr_path,'deltaT_50_off_100.mat'),'deltaT_50_off_100','lat');

### ocean, MT=off, e=0

In [33]:
sr=squeeze(solar_m(:,:,2));
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1
c_exc=0; %m/s

dt=6*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:364.9]; %date in a year

D=50*(1+.0*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[1:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

C_EXC=zeros(size(sr1,1),size(sr1,1));
ll=(lat0(1:end-1)+lat0(2:end))/2;
for m=2:size(sr1,1)-1
    C_EXC(m,m-1)=-cosd(ll(m-1));
    C_EXC(m,m)=cosd(ll(m-1))+cosd(ll(m));
    C_EXC(m,m+1)=-cosd(ll(m));
end
C_EXC(1,1)=cosd(ll(1));C_EXC(1,2)=-cosd(ll(1));
C_EXC(end,end-1)=-cosd(ll(end));C_EXC(end,end)=cosd(ll(end));
ll=cosd(lat0);ll([1,end])=dy/8/R;
C_EXC=c_exc*dt/dy*C_EXC./ll;
C_EXC=inv(eye(size(sr1,1))+C_EXC);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.4*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    Ta(:,1,m+1)=C_EXC*Ta(:,1,m+1);
    
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [34]:
aa1=squeeze(To(:,1,end-4*365+1:1:end));
aa=mean(aa1.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(.125:.25:365,1,[])/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(1/48:1/24:365,1,[])/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_50_off_000=365-2*(imax-imin);
lat=lat0;lat(35:62)=nan;

save(append(curr_path,'deltaT_50_off_000.mat'),'deltaT_50_off_000','lat');

### ocean, MT=off, omega=280

In [35]:
sr=squeeze(solar_m(:,:,1));
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1
c_exc=0; %m/s

dt=6*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:364.9]; %date in a year

D=50*(1+.0*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[1:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

C_EXC=zeros(size(sr1,1),size(sr1,1));
ll=(lat0(1:end-1)+lat0(2:end))/2;
for m=2:size(sr1,1)-1
    C_EXC(m,m-1)=-cosd(ll(m-1));
    C_EXC(m,m)=cosd(ll(m-1))+cosd(ll(m));
    C_EXC(m,m+1)=-cosd(ll(m));
end
C_EXC(1,1)=cosd(ll(1));C_EXC(1,2)=-cosd(ll(1));
C_EXC(end,end-1)=-cosd(ll(end));C_EXC(end,end)=cosd(ll(end));
ll=cosd(lat0);ll([1,end])=dy/8/R;
C_EXC=c_exc*dt/dy*C_EXC./ll;
C_EXC=inv(eye(size(sr1,1))+C_EXC);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.4*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    Ta(:,1,m+1)=C_EXC*Ta(:,1,m+1);
    
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [36]:
aa1=squeeze(To(:,1,end-4*365+1:1:end));
aa=mean(aa1.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(.125:.25:365,1,[])/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(1/48:1/24:365,1,[])/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_50_off_280=365-2*(imax-imin);
lat=lat0;lat(35:62)=nan;

save(append(curr_path,'deltaT_50_off_280.mat'),'deltaT_50_off_280','lat');

### land=2m, MT=off, omega=280

In [37]:
sr=squeeze(solar_m(:,:,1));
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1
c_exc=0; %m/s

dt=6*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:364.9]; %date in a year

D=2*(1+.0*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[1:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

C_EXC=zeros(size(sr1,1),size(sr1,1));
ll=(lat0(1:end-1)+lat0(2:end))/2;
for m=2:size(sr1,1)-1
    C_EXC(m,m-1)=-cosd(ll(m-1));
    C_EXC(m,m)=cosd(ll(m-1))+cosd(ll(m));
    C_EXC(m,m+1)=-cosd(ll(m));
end
C_EXC(1,1)=cosd(ll(1));C_EXC(1,2)=-cosd(ll(1));
C_EXC(end,end-1)=-cosd(ll(end));C_EXC(end,end)=cosd(ll(end));
ll=cosd(lat0);ll([1,end])=dy/8/R;
C_EXC=c_exc*dt/dy*C_EXC./ll;
C_EXC=inv(eye(size(sr1,1))+C_EXC);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.4*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    Ta(:,1,m+1)=C_EXC*Ta(:,1,m+1);
    
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [38]:
aa1=squeeze(To(:,1,end-4*365+1:1:end));
aa=mean(aa1.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(.125:.25:365,1,[])/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(1/48:1/24:365,1,[])/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_2_off_280=365-2*(imax-imin);
lat=lat0;lat(35:62)=nan;

save(append(curr_path,'deltaT_2_off_280.mat'),'deltaT_2_off_280','lat');

### land=2m, MT=10m/s, omega=280

In [39]:
sr=squeeze(solar_m(:,:,1));
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1
c_exc=10; %m/s

dt=6*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:364.9]; %date in a year

D=2*(1+.0*cosd(reshape(t2,1,[])*360/365-30));
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[1:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

C_EXC=zeros(size(sr1,1),size(sr1,1));
ll=(lat0(1:end-1)+lat0(2:end))/2;
for m=2:size(sr1,1)-1
    C_EXC(m,m-1)=-cosd(ll(m-1));
    C_EXC(m,m)=cosd(ll(m-1))+cosd(ll(m));
    C_EXC(m,m+1)=-cosd(ll(m));
end
C_EXC(1,1)=cosd(ll(1));C_EXC(1,2)=-cosd(ll(1));
C_EXC(end,end-1)=-cosd(ll(end));C_EXC(end,end)=cosd(ll(end));
ll=cosd(lat0);ll([1,end])=dy/8/R;
C_EXC=c_exc*dt/dy*C_EXC./ll;
C_EXC=inv(eye(size(sr1,1))+C_EXC);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    DD=round(10*D(mm));
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.4*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    Ta(:,1,m+1)=C_EXC*Ta(:,1,m+1);
    
    if(To(1,1,m+1)<0);break;end
end
m

m = 73000

In [40]:
aa1=squeeze(To(:,1,end-4*365+1:1:end));
aa=mean(aa1.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(.125:.25:365,1,[])/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(1/48:1/24:365,1,[])/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_2_10_280=365-2*(imax-imin);
lat=lat0;lat(35:62)=nan;

save(append(curr_path,'deltaT_2_10_280.mat'),'deltaT_2_10_280','lat');

### varies D&MT

In [8]:
parpool(4)

Starting parallel pool (parpool) using the 'Processes' profile ...
Connected to parallel pool with 4 workers.

ans = 

 ProcessPool with properties: 

            Connected: true
           NumWorkers: 4
                 Busy: false
              Cluster: Processes (Local Cluster)
        AttachedFiles: {}
    AutoAddClientPath: true
            FileStore: [1x1 parallel.FileStore]
           ValueStore: [1x1 parallel.ValueStore]
          IdleTimeout: 30 minutes (30 minutes remaining)
          SpmdEnabled: true



In [6]:
sr=squeeze(solar_m(:,:,1));
lat0=lat4;
R=6370000;dy=2*pi/360*abs(lat0(1)-lat0(2))*R;

beta=[0:.1:10]';
depth=[1:.1:10,11:1:20,30,40,50]';
deltaT_vary=zeros(size(beta,1),size(depth,1),size(sr,1));

parfor mdepth=1:size(depth,1)
%for mdepth=1:1%size(depth,1)

deltaT=zeros(size(beta,1),1,size(sr,1));
for mbeta=1:size(beta,1)

sigma=5.67e-8; %W/(m2*K)
Cpo=4e3;Cpa=1e3; %J/(kg*K)
rhoodz=1e2; %kg/m2--dz=10cm
ma=1e4; %kg/m2
alpha=.1;gamma=.1;epsilon=.8;SL=0; %1
c_exc=beta(mbeta); %m/s

dt=1*60*60;tt=50*365*86400/dt;
t2=[0:dt/86400:365-.001]; %date in a year

D=depth(mdepth);DD=round(10*D);
sr1=sr;
aa=mean(sr1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*t2/365));
sr1=sum(aa(:,:,1:4),3)+mean(sr1,2);

C_EXC=zeros(size(sr1,1),size(sr1,1));
ll=(lat0(1:end-1)+lat0(2:end))/2;
for m=2:size(sr1,1)-1
    C_EXC(m,m-1)=-cosd(ll(m-1));
    C_EXC(m,m)=cosd(ll(m-1))+cosd(ll(m));
    C_EXC(m,m+1)=-cosd(ll(m));
end
C_EXC(1,1)=cosd(ll(1));C_EXC(1,2)=-cosd(ll(1));
C_EXC(end,end-1)=-cosd(ll(end));C_EXC(end,end)=cosd(ll(end));
ll=cosd(lat0);ll([1,end])=dy/8/R;
C_EXC=c_exc*dt/dy*C_EXC./ll;
C_EXC=inv(eye(size(sr1,1))+C_EXC);

To=zeros(size(sr1,1),1,tt)+290;
Ta=zeros(size(sr1,1),1,tt)+290;
aa1=zeros(size(sr1,1),tt);aa2=zeros(size(sr1,1),tt);
for m=1:tt

    mm=mod(m,size(sr1,2));
    if(mm==0);mm=size(sr1,2);end
    
    aa1(:,m)=(1-alpha)*(1-gamma)*sr1(:,mm) ...
        +epsilon*sigma*Ta(:,1,m).^4-sigma*To(:,1,m).^4 ...
        -SL*(To(:,1,m)-Ta(:,1,m));
    aa2(:,m)=gamma*(1+alpha-alpha*gamma)*sr1(:,mm) ...
        +epsilon*sigma*To(:,1,m).^4-epsilon*sigma*2*Ta(:,1,m).^4 ...
        +SL*(To(:,1,m)-Ta(:,1,m));
    if(m<5.5*365*86400/dt)
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/1e7;
    else
        To(:,1,m+1)=To(:,1,m)+dt*aa1(:,m)/(DD*Cpo*rhoodz);
    end
    Ta(:,1,m+1)=Ta(:,1,m)+dt*aa2(:,m)/(Cpa*ma);
    Ta(:,1,m+1)=C_EXC*Ta(:,1,m+1);
    
    if(To(1,1,m+1)<0);break;end
end

aa1=squeeze(To(:,1,end-365*86400/dt+1:1:end));
aa=mean(aa1.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(1/48:1/24:365,1,[])/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(1/48:1/24:365,1,[])/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT(mbeta,1,:)=365-2*(imax-imin);

end

deltaT_vary(:,mdepth,:)=deltaT;
mdepth
end

Error connecting to MATLAB. Check the status of MATLAB by clicking the "Open MATLAB" button. Retry after ensuring MATLAB is running successfully

In [52]:
beta0=(beta([1,1:end])+beta([1:end,end]))/2;
beta0(1)=2*beta(1)-beta(2);
beta0(end)=2*beta(end)-beta(end-1);
depth0=(depth([1,1:end])+depth([1:end,end]))/2;
depth0(1)=2*depth(1)-depth(2);
depth0(end)=2*depth(end)-depth(end-1);

save(append(curr_path,'deltaT_vary.mat'),'deltaT_vary','beta0','depth0');

Error connecting to MATLAB. Check the status of MATLAB by clicking the "Open MATLAB" button. Retry after ensuring MATLAB is running successfully

In [1]:
delete(gcp('nocreate'));

# Global Pattern

## ERA5 delta_T 5x5

In [41]:
aa0=t2m;
aa0=reshape(aa0,size(aa0,1),size(aa0,2),[]);
tt=15;aa=0*aa0;
for m=1:2*tt+1
    aa(:,:,tt+1:end-tt)=aa(:,:,tt+1:end-tt)+aa0(:,:,m:end-(2*tt+1-m));
end
aa0=aa/(2*tt+1);
aa0=reshape(aa0,size(aa0,1),size(aa0,2),365,[]);
aa0=squeeze(mean(aa0(:,:,:,2:end-1),4));

In [42]:
air_new=zeros(72,37,365);
for m=1:72
for n=1:37
    lx=5*(m-1);ly=90-5*(n-1);
    aa=aa0(lon2>lx-2.5&lon2<lx+2.5,lat2>ly-2.5&lat2<ly+2.5,:);
    air_new(m,n,:)=mean(mean(aa,1),2);
end
end

aa=mean(air_new.*exp(1i*2*pi*reshape(1:10,1,1,1,[]) ...
    .*reshape(.5:365,1,1,[])/365),3);
aa=2*real(aa.*exp(-1i*2*pi*reshape(1:10,1,1,1,[]).*reshape(1/48:1/24:365,1,1,[])/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],3);imax=imax/24;
[~,imin]=min(aa1,[],3);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_5deg=365-2*(imax-imin);

lon=[0:5:359]';
lat=[90:-5:-90]';
deltaT_5deg(:,15:23)=nan;
save(append(curr_path,'deltaT_5deg.mat'),'deltaT_5deg','lon','lat');

## deltaT in different regions

In [43]:
aa1=squeeze(mean(aa0,1));
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:4),3);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_tt=365-2*(imax-imin);

aa1=aa0([181:231],:,:);
aa1=squeeze(mean(aa1,1));
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,1:4),3);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_epo=365-2*(imax-imin);

lat=lat2;lat(65:117)=nan;
save(append(curr_path,'deltaT_region.mat'),'deltaT_tt','deltaT_epo','lat');

## SAT Annual Cycles

In [44]:
ll=[151,211,241];
aa1=reshape(mean(mean(aa0(ll(1)-2:ll(1)+2,44:48,:),1),2),1,[]);
aa1(2,:)=reshape(mean(mean(aa0(ll(2)-2:ll(2)+2,44:48,:),1),2),1,[]);
aa1(3,:)=reshape(mean(mean(aa0(ll(3)-2:ll(3)+2,44:48,:),1),2),1,[]);
aa1=aa1-mean(aa1,2);
[~,imax]=max(aa1,[],2);
[~,imin]=min(aa1,[],2);

save(append(curr_path,'ac_45.mat'),'aa1','imax','imin');

## CESM2

In [45]:
aa0=t2m_m; %1980-2020
aa0=reshape(aa0,size(aa0,1),size(aa0,2),[],3);
tt=15;aa=0*aa0;
for m=1:2*tt+1
    aa(:,:,tt+1:end-tt,:)=aa(:,:,tt+1:end-tt,:)+aa0(:,:,m:end-(2*tt+1-m),:);
end
aa0=aa/(2*tt+1);
aa0=reshape(aa0,size(aa0,1),size(aa0,2),365,[],3);
aa0=squeeze(mean(aa0(:,:,:,2:end-1,:),4));

In [46]:
aa1=squeeze(mean(aa0,1));
aa=mean(aa1.*exp(1i*2*pi*reshape([1:10],1,1,1,[]).*[.5:365]/365),2);
aa=2*real(aa.*exp(-1i*2*pi*reshape([1:10],1,1,1,[]).*[1/48:1/24:365]/365));
aa1=sum(aa(:,:,:,1:4),4);
[~,imax]=max(aa1,[],2);imax=imax/24;
[~,imin]=min(aa1,[],2);imin=imin/24;
imax(imax<imin)=imax(imax<imin)+365;
deltaT_cesm=squeeze(365-2*(imax-imin));

lat=lat4;lat(35:62)=nan;
save(append(curr_path,'deltaT_cesm.mat'),'deltaT_cesm','lat');

## Sea Ice

In [47]:
aa1=sic;
aa1(isnan(aa1))=0;
sic_std=mean(std(mean(aa1,4),1,3),1)';

lat=lat2;
save(append(curr_path,'sic_std.mat'),'sic_std','lat');